CODE 1: LANDMARK POINTS, GAUSSIAN BLUR AND EDGE DETECTION
https://github.com/siddh30/Glasses-Detection/blob/main/Glasses_detection.ipynb

In [1]:
import dlib
import cv2
import numpy as np
import os

lfw_dataset_path = "/Users/nsjain/Downloads/lfw"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/nsjain/Downloads/shape_predictor_68_face_landmarks.dat")

def glasses_detector(image_path):
    img = dlib.load_rgb_image(image_path)
    
    if len(detector(img)) == 0:
        return 'No face detected'
    
    rect = detector(img)[0]
    sp = predictor(img, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])
    
    nose_bridge_x = []
    nose_bridge_y = []
    
    for i in [28, 29, 30, 31, 33, 34, 35]:
        nose_bridge_x.append(landmarks[i][0])
        nose_bridge_y.append(landmarks[i][1])
    
    x_min = min(nose_bridge_x)
    x_max = max(nose_bridge_x)
    
    y_min = landmarks[20][1]
    y_max = landmarks[30][1]
    
    img2 = cv2.imread(image_path)
    img2 = img2[y_min:y_max, x_min:x_max]
    
    img_blur = cv2.GaussianBlur(img2, (3, 3), sigmaX=0, sigmaY=0)
    
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)
    
    edges_center = edges.T[(int(len(edges.T)/2))]
    
    if 255 in edges_center:
        return 1
    else:
        return 0

glasses_detected_count = 0
for root, dirs, files in os.walk(lfw_dataset_path):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            result = glasses_detector(image_path)
            
            if result == 1:
                glasses_detected_count += 1
            
print(f"Total images with glasses detected: {glasses_detected_count}")

Total images with glasses detected: 6740


CODE 2: LANDMARK POINTS, ALIGNMENT, GAUSSIAN BLUR AND EDGE DETECTION
https://github.com/TianxingWu/realtime-glasses-detection/blob/master/eyeglass_detector.py

In [16]:
import dlib
import cv2
import numpy as np
import os

lfw_dataset_path = "/Users/nsjain/Desktop/noglasses"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/nsjain/Downloads/shape_predictor_68_face_landmarks.dat")

def landmarks_to_np(landmarks, dtype="int"):
    num = landmarks.num_parts
    coords = np.zeros((num, 2), dtype=dtype)
    for i in range(0, num):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
    return coords

def get_centers(img, landmarks):
    EYE_LEFT_OUTTER = landmarks[2]
    EYE_LEFT_INNER = landmarks[3]
    EYE_RIGHT_OUTTER = landmarks[0]
    EYE_RIGHT_INNER = landmarks[1]

    x = ((landmarks[0:4]).T)[0]
    y = ((landmarks[0:4]).T)[1]
    A = np.vstack([x, np.ones(len(x))]).T
    k, b = np.linalg.lstsq(A, y, rcond=None)[0]

    x_left = (EYE_LEFT_OUTTER[0]+EYE_LEFT_INNER[0])/2
    x_right = (EYE_RIGHT_OUTTER[0]+EYE_RIGHT_INNER[0])/2
    LEFT_EYE_CENTER =  np.array([np.int32(x_left), np.int32(x_left*k+b)])
    RIGHT_EYE_CENTER =  np.array([np.int32(x_right), np.int32(x_right*k+b)])

    pts = np.vstack((LEFT_EYE_CENTER,RIGHT_EYE_CENTER))
    cv2.polylines(img, [pts], False, (255,0,0), 1)
    cv2.circle(img, (LEFT_EYE_CENTER[0],LEFT_EYE_CENTER[1]), 3, (0, 0, 255), -1)
    cv2.circle(img, (RIGHT_EYE_CENTER[0],RIGHT_EYE_CENTER[1]), 3, (0, 0, 255), -1)

    return LEFT_EYE_CENTER, RIGHT_EYE_CENTER

def get_aligned_face(img, left, right):
    desired_w = 256
    desired_h = 256
    desired_dist = desired_w * 0.5

    eyescenter = ((left[0]+right[0])*0.5 , (left[1]+right[1])*0.5)
    dx = right[0] - left[0]
    dy = right[1] - left[1]
    dist = np.sqrt(dx*dx + dy*dy)
    scale = desired_dist / dist
    angle = np.degrees(np.arctan2(dy,dx))
    M = cv2.getRotationMatrix2D(eyescenter,angle,scale)

    tX = desired_w * 0.5
    tY = desired_h * 0.5
    M[0, 2] += (tX - eyescenter[0])
    M[1, 2] += (tY - eyescenter[1])

    aligned_face = cv2.warpAffine(img,M,(desired_w,desired_h))

    return aligned_face

def judge_eyeglass(img):
    img = cv2.GaussianBlur(img, (11,11), 0)

    sobel_y = cv2.Sobel(img, cv2.CV_64F, 0 ,1 , ksize=-1)
    sobel_y = cv2.convertScaleAbs(sobel_y)
    cv2.imshow('sobel_y', sobel_y)

    edgeness = sobel_y

    retVal, thresh = cv2.threshold(edgeness, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    d = len(thresh) * 0.5
    x = np.int32(d * 6/7)
    y = np.int32(d * 3/4)
    w = np.int32(d * 2/7)
    h = np.int32(d * 2/4)

    x_2_1 = np.int32(d * 1/4)
    x_2_2 = np.int32(d * 5/4)
    w_2 = np.int32(d * 1/2)
    y_2 = np.int32(d * 8/7)
    h_2 = np.int32(d * 1/2)

    roi_1 = thresh[y:y+h, x:x+w]
    roi_2_1 = thresh[y_2:y_2+h_2, x_2_1:x_2_1+w_2]
    roi_2_2 = thresh[y_2:y_2+h_2, x_2_2:x_2_2+w_2]
    roi_2 = np.hstack([roi_2_1,roi_2_2])

    measure_1 = sum(sum(roi_1/255)) / (np.shape(roi_1)[0] * np.shape(roi_1)[1])
    measure_2 = sum(sum(roi_2/255)) / (np.shape(roi_2)[0] * np.shape(roi_2)[1])
    measure = measure_1*0.3 + measure_2*0.7

    cv2.imshow('roi_1', roi_1)
    cv2.imshow('roi_2', roi_2)

    if measure > 0.15:
        judge = True
    else:
        judge = False
    return judge

glasses_detected_count = 0
for root, dirs, files in os.walk(lfw_dataset_path):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
            
            for face in faces:
                landmarks = predictor(gray, face)
                landmarks = landmarks_to_np(landmarks)
                left_eye_center, right_eye_center = get_centers(img, landmarks)
                aligned_face = get_aligned_face(gray, left_eye_center, right_eye_center)
                is_wearing_glasses = judge_eyeglass(aligned_face)

                if is_wearing_glasses:
                    glasses_detected_count += 1
                    #print(f"{image_path}: Glasses detected")
            
print(f"Total images with glasses detected: {glasses_detected_count}")

/var/folders/34/j8dqnjr14_7d5fwpn4wx4zm80000gq/T/ipykernel_33606/3458529870.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  scale = desired_dist / dist


Total images with glasses detected: 283


CODE 3: LANDMARK POINTS, IMAGE AUGMENTATION, TRAIN USING CNN ON LFW DATASET

In [1]:
import dlib
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tqdm import tqdm


lfw_dataset_path = "/Users/nsjain/Downloads/lfw"
shape_predictor_path = "/Users/nsjain/Downloads/shape_predictor_68_face_landmarks.dat"

IMG_WIDTH, IMG_HEIGHT = 128, 128
GLASSES_LABEL = 1

np.random.seed(42)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img.astype('float32') / 255.0
    return img

def extract_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = detector(gray)
    landmarks = []
    for face in faces:
        shape = predictor(gray, face)
        for i in range(68):
            landmarks.append((shape.part(i).x, shape.part(i).y))
    return landmarks

lfw_images = []
lfw_labels = []
total_images = 0

progress_bar = tqdm(total=total_images, desc="Loading images", unit="image")

for root, dirs, files in os.walk(lfw_dataset_path):
    total_images += len([file for file in files if file.endswith(".jpg")])

for root, dirs, files in tqdm(os.walk(lfw_dataset_path), desc="Loading images", unit="image"):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            lfw_images.append(preprocess_image(image_path))
            lfw_labels.append(GLASSES_LABEL if 'glasses' in file else 0)
            progress_bar.update(1) 

progress_bar.close()

X_train, X_test, y_train, y_test = train_test_split(lfw_images, lfw_labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

progress_bar = tqdm(total=len(X_train), desc="Training model", unit="image")
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), verbose=2)
progress_bar.close() 

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Loading images: 5750image [00:06, 914.40image/s]s]
Loading images: 13232image [00:06, 2047.46image/s]
Training model:   0%|          | 0/10585 [00:00<?, ?image/s]

Epoch 1/10
331/331 - 38s - loss: 0.0022 - accuracy: 0.9979 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 38s/epoch - 114ms/step
Epoch 2/10
331/331 - 37s - loss: 1.7746e-24 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 37s/epoch - 112ms/step
Epoch 3/10
331/331 - 38s - loss: 8.0342e-26 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 38s/epoch - 113ms/step
Epoch 4/10
331/331 - 38s - loss: 2.9713e-26 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 38s/epoch - 113ms/step
Epoch 5/10
331/331 - 37s - loss: 5.6584e-24 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 37s/epoch - 112ms/step
Epoch 6/10
331/331 - 37s - loss: 2.2016e-27 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 37s/epoch - 113ms/step
Epoch 7/10
331/331 - 38s - loss: 5.7568e-26 - accuracy: 1.0000 - val_loss: 9.5215e-32 - val_accuracy: 1.0000 - 38s/epoch - 113ms/step
Epoch 8/10
331/331 - 37s - loss: 9.5753e-28 - accuracy: 1.0000 - v

Training model:   0%|          | 0/10585 [06:13<?, ?image/s]


83/83 [==============================] - 2s 29ms/step - loss: 9.5215e-32 - accuracy: 1.0000
Test Loss: 9.521530690283422e-32
Test Accuracy: 1.0


In [2]:
sample_image_path = "/Users/nsjain/Downloads/lfw/Abba_Eban/Abba_Eban_0001.jpg"
sample_image = preprocess_image(sample_image_path)

sample_image = np.expand_dims(sample_image, axis=0)

predictions = model.predict(sample_image)

if predictions[0] >= 0.5:
    print("Glasses detected")
else:
    print("No glasses detected")

1/1 [==============================] - 0s 67ms/step
No glasses detected


In [3]:
sample_image_path = "/Users/nsjain/Downloads/lfw/Aaron_Sorkin/Aaron_Sorkin_0001.jpg"
sample_image = preprocess_image(sample_image_path)

sample_image = np.expand_dims(sample_image, axis=0)

predictions = model.predict(sample_image)

if predictions[0] >= 0.5:
    print("Glasses detected")
else:
    print("No glasses detected")

1/1 [==============================] - 0s 17ms/step
No glasses detected


CODE 4: CREATED TWO DATASETS OF GLASSES AND NON GLASSES AND IMPLEMENTED THE SAME CODE AS ABOVE

In [6]:
import dlib
import cv2
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

noglasses_dataset_path = "/Users/nsjain/Desktop/noglasses"
shape_predictor_path = "/Users/nsjain/Downloads/shape_predictor_68_face_landmarks.dat"
glasses_images_path = "/Users/nsjain/Desktop/glasses"

IMG_WIDTH, IMG_HEIGHT = 128, 128
GLASSES_LABEL = 1

random.seed(42)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img.astype('float32') / 255.0
    return img

def extract_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = detector(gray)
    landmarks = []
    for face in faces:
        shape = predictor(gray, face)
        for i in range(68):
            landmarks.append((shape.part(i).x, shape.part(i).y))
    return landmarks

def generate_augmented_images(images, labels):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    augmented_images = []
    augmented_labels = []
    for image, label in zip(images, labels):
        image = np.expand_dims(image, axis=0)
        aug_iter = datagen.flow(image, batch_size=1)
        for _ in range(5): 
            augmented_image = aug_iter.next()[0]
            augmented_images.append(augmented_image)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

glasses_images = []
glasses_labels = []
for root, dirs, files in os.walk(glasses_images_path):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            glasses_images.append(preprocess_image(image_path))
            glasses_labels.append(GLASSES_LABEL)

lfw_images = []
lfw_labels = []
for root, dirs, files in os.walk(lfw_dataset_path):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            lfw_images.append(preprocess_image(image_path))
            lfw_labels.append(0)

images = glasses_images + lfw_images
labels = glasses_labels + lfw_labels

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


X_train_aug, y_train_aug = generate_augmented_images(X_train, y_train)

X_train_final = np.concatenate((X_train, X_train_aug), axis=0)
y_train_final = np.concatenate((y_train, y_train_aug), axis=0)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

model.fit(X_train_final, y_train_final, batch_size=32, epochs=10, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


Epoch 1/10
158/158 [==============================] - 18s 111ms/step - loss: 0.5635 - accuracy: 0.7544 - val_loss: 0.5466 - val_accuracy: 0.7441
Epoch 2/10
158/158 [==============================] - 17s 109ms/step - loss: 0.5197 - accuracy: 0.7573 - val_loss: 0.4954 - val_accuracy: 0.7583
Epoch 3/10
158/158 [==============================] - 17s 110ms/step - loss: 0.4754 - accuracy: 0.7722 - val_loss: 0.5032 - val_accuracy: 0.7867
Epoch 4/10
158/158 [==============================] - 17s 108ms/step - loss: 0.4518 - accuracy: 0.7833 - val_loss: 0.4911 - val_accuracy: 0.7820
Epoch 5/10
158/158 [==============================] - 17s 107ms/step - loss: 0.4164 - accuracy: 0.8021 - val_loss: 0.4706 - val_accuracy: 0.7820
Epoch 6/10
158/158 [==============================] - 17s 107ms/step - loss: 0.3766 - accuracy: 0.8197 - val_loss: 0.5068 - val_accuracy: 0.7820
Epoch 7/10
158/158 [==============================] - 17s 107ms/step - loss: 0.3495 - accuracy: 0.8385 - val_loss: 0.5237 - val_ac

In [26]:
from sklearn.metrics import confusion_matrix
from scipy.optimize import brentq
from scipy.interpolate import interp1d

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

fpr, tpr, _ = roc_curve(y_test, y_pred)

eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
print(f"Equal Error Rate (EER): {eer}")
print("False Positive Rate:", fpr)
print("True Positive Rate:", tpr)

7/7 [==============================] - 0s 27ms/step
Confusion Matrix:
[[149   8]
 [ 31  23]]
EER: 0.3769, FAR: 1.0000, FRR: 0.0000


In [10]:
#With glasses dataset
num_images = len(glasses_images)
num_labels = len(glasses_labels)
print("Number of Images:", num_images)
print("Number of Labels:", num_labels)

Number of Images: 260
Number of Labels: 260


In [29]:
X_test = np.array(glasses_images)
y_test = np.array(glasses_labels)

# Perform any required data transformations or scaling on X_test

# Make predictions using your model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate EER, FAR, and FRR
fpr, tpr, _ = roc_curve(y_test, y_pred)

"""
eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
far = fpr[np.nanargmin(np.abs(fpr - 1. - eer))]
frr = 1 - tpr[np.nanargmin(np.abs(fpr - 1. - eer))]
print(f'EER: {eer:.4f}, FAR: {far:.4f}, FRR: {frr:.4f}')
"""

9/9 [==============================] - 0s 30ms/step - loss: 0.7190 - accuracy: 0.7769
Test Loss: 0.7189658880233765
Test Accuracy: 0.7769230604171753
Confusion Matrix:
[[  0   0]
 [ 58 202]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


"\neer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)\nfar = fpr[np.nanargmin(np.abs(fpr - 1. - eer))]\nfrr = 1 - tpr[np.nanargmin(np.abs(fpr - 1. - eer))]\nprint(f'EER: {eer:.4f}, FAR: {far:.4f}, FRR: {frr:.4f}')\n"

In [11]:
#Without glasses dataset
lfw_images = len(lfw_images)
lfw_labels = len(lfw_labels)
print("Number of Images:", lfw_images)
print("Number of Labels:", lfw_labels)

Number of Images: 793
Number of Labels: 793


In [30]:
X_test = np.array(lfw_images)
y_test = np.array(lfw_labels)

# Perform any required data transformations or scaling on X_test

# Make predictions using your model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate EER, FAR, and FRR
fpr, tpr, _ = roc_curve(y_test, y_pred)

"""
eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
far = fpr[np.nanargmin(np.abs(fpr - 1. - eer))]
frr = 1 - tpr[np.nanargmin(np.abs(fpr - 1. - eer))]
print(f'EER: {eer:.4f}, FAR: {far:.4f}, FRR: {frr:.4f}')
"""

IndexError: tuple index out of range

In [7]:
sample_image_path = "/Users/nsjain/Downloads/lfw/Abba_Eban/Abba_Eban_0001.jpg"
sample_image = preprocess_image(sample_image_path)

sample_image = np.expand_dims(sample_image, axis=0)

predictions = model.predict(sample_image)

if predictions[0] >= 0.5:
    print("Glasses detected")
else:
    print("No glasses detected")

1/1 [==============================] - 0s 42ms/step
Glasses detected


In [8]:
sample_image_path = "/Users/nsjain/Downloads/lfw/Aaron_Sorkin/Aaron_Sorkin_0001.jpg"
sample_image = preprocess_image(sample_image_path)

sample_image = np.expand_dims(sample_image, axis=0)

predictions = model.predict(sample_image)

if predictions[0] >= 0.5:
    print("Glasses detected")
else:
    print("No glasses detected")

1/1 [==============================] - 0s 16ms/step
No glasses detected


CODE 5: AMAZON REKOGNITION API

In [1]:
import boto3

rekognition_client = boto3.client('rekognition', region_name='us-east-1')

import os

lfw_dataset_path = "/Users/nsjain/Downloads/lfw"

for root, dirs, files in os.walk(lfw_dataset_path):
    for file in tqdm(files, desc="Processing files", unit="file"):
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            img = Image.open(image_path)
            img = img.resize((128, 128))
            if img.mode != "RGB":
                img = img.convert("RGB")

            with io.BytesIO() as output:
                img.save(output, format="JPEG")
                image_bytes = output.getvalue()

            response = rekognition_client.detect_faces(
                Image={'Bytes': image_bytes},
                Attributes=['ALL']
            )
            
            for face_detail in response['FaceDetails']:
                if face_detail['Eyeglasses']['Value']:
                    print(f"Glasses detected in {image_path}")

NoCredentialsError: Unable to locate credentials

OPENCV HAAR CASCADE PRE_TRAINED CLASSIFIER

In [38]:
import cv2
import os

eye_cascade = cv2.CascadeClassifier('/Users/nsjain/Downloads/haarcascade_eye.xml')
glasses_cascade = cv2.CascadeClassifier('/Users/nsjain/Downloads/haarcascade_eye_tree_eyeglasses.xml')

lfw_path = '/Users/nsjain/Desktop/glasses'

glasses_count = 0
image_count = 0

for root, dirs, files in os.walk(lfw_path):
    for file in files:
        if file.endswith(".jpg"):
            image_count += 1
            image_path = os.path.join(root, file)

            image = cv2.imread(image_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (ex, ey, ew, eh) in eyes:
                roi_gray = gray[ey:ey+eh, ex:ex+ew]

                # Perform glasses detection within the eye region
                glasses = glasses_cascade.detectMultiScale(roi_gray, scaleFactor=1.05, minNeighbors=3, minSize=(10, 10))

                if len(glasses) > 0:
                    glasses_count += 1

        """
        # Draw rectangles around the detected eyes
        for (x, y, w, h) in eyes:
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Draw rectangles around the detected eyeglasses
        for (x, y, w, h) in glasses:
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Display the image with the detections
        cv2.imshow('Image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        """
        
print(f"Number of images with detected glasses: {glasses_count}")
print("Total images with glasses:", image_count)

Number of images with detected glasses: 91
Total images with glasses: 260


TENSORFLOW OBJECT DETECTION API

In [ ]:
import tensorflow as tf
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import cv2

# Path to the frozen inference graph and label map
PATH_TO_FROZEN_GRAPH = '/path/to/frozen_inference_graph.pb'
PATH_TO_LABEL_MAP = '/path/to/label_map.pbtxt'

# Load the frozen inference graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Load the label map
label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=2, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Function to perform object detection on an image
def detect_glasses(image):
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            # Preprocess the image
            image_np_expanded = np.expand_dims(image, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            # Perform inference
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            # Visualize the results
            vis_util.visualize_boxes_and_labels_on_image_array(
                image,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4,
                min_score_thresh=0.5)

            return image

# Path to your dataset directory
dataset_path = '/path/to/dataset'

glasses_detected_count = 0
total_images = 0

# Process each image in the dataset
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.jpg'):
            total_images += 1
            image_path = os.path.join(root, file)
            image = cv2.imread(image_path)
            image_with_glasses = detect_glasses(image)

            # Count the number of images with detected glasses
            if np.any(image_with_glasses):
                glasses_detected_count += 1

# Print the results
print("Number of images with detected glasses:", glasses_detected_count)
print("Total images with glasses:", total_images)


INSIGHTFACE

In [48]:
import insightface
from insightface.app import FaceAnalysis

model = FaceAnalysis()
model.prepare(ctx_id=-1)

detector = insightface.app.FaceDetector()
detector.prepare(ctx_id=-1, nms=0.4)

def align_face(image, bbox):
    landmarks = model.get_landmarks(image, bbox)
    return insightface.utils.face_align.norm_crop(image, landmarks)

def detect_glasses(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = model.get(image)
    
    glasses_count = 0
    
    for idx, face in enumerate(faces):
        if face.landmark is not None:
            landmarks = face.landmark.astype(int)
            
            left_eye = landmarks[36:42]
            right_eye = landmarks[42:48]
            
            avg_left_eye = np.mean(left_eye, axis=0)
            avg_right_eye = np.mean(right_eye, axis=0)
            
            distance = np.linalg.norm(avg_left_eye - avg_right_eye)
            if distance > 10:
                glasses_count += 1
    
    return glasses_count


lfw_dataset_path = "//Users/nsjain/Desktop/glasses"

glasses_detected_count = 0
total_images = 0

for root, dirs, files in os.walk(lfw_dataset_path):
    for file in files:
        if file.endswith(".jpg"):
            total_images += 1
            image_path = os.path.join(root, file)
            
            glasses_count = detect_glasses(image_path)
            
            if glasses_count > 0:
                glasses_detected_count += 1

print("Number of images with detected glasses:", glasses_detected_count)
print("Total images with glasses:", total_images)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/nsjain/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/nsjain/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/nsjain/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/nsjain/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/nsjain/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

AttributeError: module 'insightface.app' has no attribute 'FaceDetector'